In [75]:
import string
from datetime import datetime
import os
import numpy as np
import pandas as pd
import json
from collections import defaultdict

import nltk
# nltk.data.path.append("/")

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from gensim.models.word2vec import Word2Vec
from gensim.utils import simple_preprocess
from gensim.models.doc2vec import TaggedDocument
from gensim.models.doc2vec import Doc2Vec

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.metrics.pairwise import cosine_similarity

In [118]:
import plotly
plotly.tools.set_credentials_file(username='amcire96', api_key='sej35ud4YbSOfIshhhZg')
# print(plotly.__version__)

import plotly.plotly as py
from plotly.graph_objs import *
import requests
requests.packages.urllib3.disable_warnings()

In [5]:
print(234)

234


In [8]:
reviews = []
with open("yelp_academic_dataset_review.json", "r") as f:
    for line in f:
        json_line = json.loads(line)
        reviews.append(json_line)

In [36]:
print(len(reviews))
reviews[0]

4153150


{u'business_id': u'2aFiy99vNLklCx3T_tGS9A',
 u'cool': 0,
 u'date': u'2011-10-10',
 u'funny': 0,
 u'review_id': u'NxL8SIC5yqOdnlXCg18IBg',
 u'stars': 5,
 u'text': u"If you enjoy service by someone who is as competent as he is personable, I would recommend Corey Kaplan highly. The time he has spent here has been very productive and working with him educational and enjoyable. I hope not to need him again (though this is highly unlikely) but knowing he is there if I do is very nice. By the way, I'm not from El Centro, CA. but Scottsdale, AZ.",
 u'type': u'review',
 u'useful': 0,
 u'user_id': u'KpkOkG6RIf4Ra25Lhhxf1A'}

In [9]:
businesses = []
with open("yelp_academic_dataset_business.json", "r") as f:
    for line in f:
        json_line = json.loads(line)
        businesses.append(json_line)

In [15]:
print(len(businesses))
businesses[0]

144072


[{u'address': u'227 E Baseline Rd, Ste J2',
  u'attributes': [u'BikeParking: True',
   u'BusinessAcceptsBitcoin: False',
   u'BusinessAcceptsCreditCards: True',
   u"BusinessParking: {'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}",
   u'DogsAllowed: False',
   u'RestaurantsPriceRange2: 2',
   u'WheelchairAccessible: True'],
  u'business_id': u'0DI8Dt2PJp07XkVvIElIcQ',
  u'categories': [u'Tobacco Shops', u'Nightlife', u'Vape Shops', u'Shopping'],
  u'city': u'Tempe',
  u'hours': [u'Monday 11:0-21:0',
   u'Tuesday 11:0-21:0',
   u'Wednesday 11:0-21:0',
   u'Thursday 11:0-21:0',
   u'Friday 11:0-22:0',
   u'Saturday 10:0-22:0',
   u'Sunday 11:0-18:0'],
  u'is_open': 0,
  u'latitude': 33.3782141,
  u'longitude': -111.936102,
  u'name': u'Innovative Vapors',
  u'neighborhood': u'',
  u'postal_code': u'85283',
  u'review_count': 17,
  u'stars': 4.5,
  u'state': u'AZ',
  u'type': u'business'}]

In [84]:
valid_business_ids = set()
business_id_to_index = {}
business_index_to_bid = {}
c = 0
for business in businesses:
    if business["review_count"] >= 1000:
        valid_business_ids.add(business["business_id"])
        if business["business_id"] not in business_id_to_index:
            business_id_to_index[business["business_id"]] = c
            business_index_to_bid[c] = business["business_id"]
            c += 1
print(len(valid_business_ids))
valid_business_indices

177


{u'--9e1ONYQuAa-CB_Rrw7Tw': 92,
 u'-ed0Yc9on37RoIoG2ZgxBA': 109,
 u'0FUtlsQrJI7LhqDPxLumEw': 106,
 u'0NmTwqYEQiKErDv4a55obg': 161,
 u'0W4lkclzZThpx3V65bVgig': 21,
 u'2iTsRqUsPGRH1li1WVRvKQ': 112,
 u'2weQS-RnoOBhb1KsHKyoSQ': 75,
 u'3BCsAgo_1i4xMuTyLKMLRQ': 6,
 u'3GEEy7RP6e4bT4LAiWFMFQ': 44,
 u'3N9U549Zse8UP-MwKZAjAQ': 130,
 u'3kdSl5mo9dWC4clrQjEDGg': 137,
 u'3l54GTr8-E3XPbIxnF_sAA': 53,
 u'3xmfT7l3xNH5LK1dLzfvGw': 58,
 u'4GXII-GU7S0ZyU6ElkhscQ': 144,
 u'4JNXUYY8wbaaDmk3BPzlWw': 12,
 u'4k3RlMAMd46DZ_JyZU0lMg': 3,
 u'5LNZ67Yw9RD6nf4_UhXOjw': 108,
 u'5T6kFKFycym_GkhgOiysIw': 151,
 u'5iSmZO0SrKU6EoXK_1M8Kw': 1,
 u'5shgJB7a-2_gdnzc0gsOtg': 42,
 u'6Ct57qgmXwOnzfSZoUGh0Q': 45,
 u'6Q7-wkCPc1KF75jZLOTcMw': 96,
 u'7fxebHYUwIF6CakxSr70iQ': 51,
 u'7sPNbCx7vGAaH7SbNPZ6oA': 146,
 u'7sb2FYLS2sejZKxRYF9mtg': 140,
 u'9SU7ZZhaFUJJ6m2k5HKHeg': 174,
 u'9a3DrZvpYxVs3k_qwlCNSw': 13,
 u'A-uZAD4zP3rRxb44WUGV5w': 32,
 u'A5Rkh7UymKm0_Rxm9K2PJw': 147,
 u'AV6weBrZFFBfRGCbcRGO4g': 173,
 u'Az_60nNuh1FH8Ds8oasZjw': 6

In [59]:
only_text = []
ordered_review_ids = []
ordered_business_id_indices = []
bid_to_rid_dict = defaultdict(list)
rid_to_bid_dict = {}
for review in reviews:
    if review["business_id"] in valid_business_ids:
        only_text.append(review["text"])
        ordered_review_ids.append(review["review_id"])
        ordered_business_id_indices.append(valid_business_indices[review["business_id"]])
        bid_to_rid_dict[review["business_id"]].append(review["review_id"])
        rid_to_bid_dict[review["review_id"]] = review["business_id"]
print(len(only_text))
print(len(ordered_review_ids))

315002
315002


In [29]:

tfidf_vectorizer = TfidfVectorizer(input=only_text, lowercase=True, stop_words="english", max_df=0.9, min_df=100)
tfidf_review_text = tfidf_vectorizer.fit_transform(only_text)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

print(tfidf_review_text.shape)

(315002, 9071)


In [30]:
count_vectorizer = CountVectorizer(input=only_text, lowercase=True, stop_words="english", max_df=0.9, min_df=100)
count_review_text = count_vectorizer.fit_transform(only_text)
count_feature_names = count_vectorizer.get_feature_names()

print(count_review_text.shape)

(315002, 9071)


In [ ]:
print(count_feature_names)

In [27]:
def display_topics(model, feature_names, n_top_words):
    for topic_index, topic in enumerate(model.components_):
        print("Topic %d:" % topic_index)
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words:-1]]))

In [33]:
lda_model = LatentDirichletAllocation(n_topics=30).fit(count_review_text)
display_topics(lda_model, count_feature_names, 20)

/home/amcire/anaconda2/lib/python2.7/site-packages/sklearn/decomposition/online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


Topic 0:
thai bbq pork rice spicy noodles marquee bf chili die encore tofu cheesecake lo el belly pulled foods acrobatics
Topic 1:
service station money bathroom worst customer horrible water paying happened terrible star employees cup bad toilet cafe caesars poor
Topic 2:
line wait minutes time 30 people 10 waiting hour night 20 told reservation 15 check went hours day waited
Topic 3:
cold quickly nights issue shopping spice ami tao nicer accommodating 30pm lovely kinds send jam welcoming wi fi prawns
Topic 4:
burger burgers gelato shake gordon bobby truffle mon gabi ramsay flay rings beer bun baby shack try cheese pub
Topic 5:
free aria fountains business buy monte carlo yes simply exceptional hallway outstanding travel street times year air gambling hands
Topic 6:
crab sushi seafood buffets fresh eat legs shrimp desserts worth rolls rib dessert fish asian roll section like hell
Topic 7:
pizza tacos oysters fresh taco delicious phoenix patio enjoyed bloody crust french frites olives 

In [34]:
nmf_model = NMF(n_components=30).fit(tfidf_review_text)
display_topics(nmf_model, tfidf_feature_names, 20)

Topic 0:
just like didn got don said came did table know minutes asked told order people wasn took went bad
Topic 1:
room check bed stay desk bathroom shower view stayed suite floor night booked tv tower called clean day resort
Topic 2:
burger fries burgers cheese shake truffle sweet onion ordered potato bun rings shack bacon shakes gordon farm medium patty
Topic 3:
great service atmosphere awesome beer selection experience location fun happy spot fantastic prices drinks loved bar wonderful wine patio
Topic 4:
buffet crab buffets legs selection seafood desserts prime wynn quality dessert station rib price variety dinner food wicked bellagio
Topic 5:
place try eat looking like want packed friends people cool places don fun town busy awesome say menu small
Topic 6:
wait line worth long hour minutes waiting 30 seated waited hours 45 lines table 20 15 pan min minute
Topic 7:
good pretty price overall beer little selection decent bit ok prices got thing bad pricey bbq wasn value taste
Topic

In [47]:
tagged_doc_list = []
for i in range(len(only_text)):
    text = only_text[i]
    tokens = nltk.word_tokenize(text)
    label = ordered_business_id_indices[i]
    tagged_doc = TaggedDocument(tokens, [label])
    tagged_doc_list.append(tagged_doc)

In [48]:
tagged_doc_list[0]

TaggedDocument(words=[u'I', u'love', u'sushi', u'!', u'Especially', u'with', u'cream', u'cheese', u'!', u'This', u'place', u'hardly', u'had', u'any', u'cream', u'cheese', u'sushi', u'but', u'it', u'is', u'still', u'overall', u'YUM', u'!', u':', u')', u'Service', u'is', u'excellent', u',', u'you', u'never', u'go', u'thirsty', u'as', u'they', u'keep', u'your', u'drink', u'constantly', u'full', u'!', u'Dinner', u'AYCE', u'is', u'around', u'$', u'26', u',', u'while', u'I', u"'ve", u'seen', u'lower', u'prices', u'.', u'I', u'tried', u'the', u'Philadelphia', u'roll', u',', u'caterpillar', u'and', u'Dragon', u'roll', u'.', u'philadelphia', u'roll', u'for', u'some', u'reason', u'was', u"n't", u'as', u'tasty', u'as', u'I', u'thought', u'.', u'Any', u'eel', u'in', u'the', u'rolls', u'I', u'have', u'tried', u'are', u'not', u'as', u'good', u'to', u'brag', u'about', u'honestly', u'.', u'I', u'give', u'4', u'stars', u'for', u'the', u'great', u'service', u',', u'and', u'it', u"'s", u'enough', u'to', 

In [45]:
only_text[0]

u"I love sushi! Especially with cream cheese! This place hardly had any cream cheese sushi but it is still overall YUM! :) Service is excellent, you never go thirsty as they keep your drink constantly full!\n\nDinner AYCE is around $26, while I've seen lower prices. I tried the Philadelphia roll, caterpillar and Dragon roll. philadelphia roll for some reason wasn't as tasty as I thought. Any eel in the rolls I have tried are not as good to brag about honestly. I give 4 stars for the great service, and it's enough to go back and give them another chance."

In [51]:
model = Doc2Vec(size=500, min_count=100, iter=5)
model.build_vocab(tagged_doc_list)
%time model.train(tagged_doc_list)

CPU times: user 12min 41s, sys: 9.85 s, total: 12min 51s
Wall time: 4min 40s


171043650

In [56]:
model.save('reviews_doc2vec_model.doc2vec')


In [57]:
# load the model back
model_loaded = Doc2Vec.load('reviews_doc2vec_model.doc2vec')

In [66]:
example_index = 100

In [69]:
example_bid = rid_to_bid_dict[ordered_review_ids[example_index]]
example_rids_for_business = bid_to_rid_dict[example_bid]

In [88]:
print(len(example_rids_for_business))
example_bid

1502


u'JyxHvtj-syke7m9rbza7mA'

In [64]:

print("Review Text is:")
print(only_text[example_index])

for rid in example_rids_for_business[:5]:
    ind = ordered_review_ids.index(rid)
    print
    print("---------------")
    print("Review index %d is:" % ind)
    print(only_text[ind])

Review Text is:
Have been coming here for past year or so
great AYCE sushi place not too far from where I live

used to frequent Sushi Factory, one of the first AYCE places in town, but sadly they closed :-(

in any case, have tried a few other sushi places, have found this one to be one of the better choices in the area

usually pretty busy, never been there without having to wait...boo

their selection is pretty good, fish is fresh, haven't really had any bad meals to date
I usually try to sit at sushi bar unless with more than a couple friends, as you can usually get some good choices/tips from sushi chefs working that day

I usually stick to nigiri, so not a whole lot to explain other that the fish is usually fresh, and the ratio to rice pretty good too....make sure to check out board for specials!

love that they have salmon collar too! I do order a few of the cooked items, such as that, maybe some of the grilled specials, and oh yeah, pork belly, YUM!

I like their tuna on crispy

In [95]:
inferred = model.infer_vector(nltk.word_tokenize(only_text[100]))
inferred

array([  4.77143005e-02,   8.92363265e-02,   4.87764239e-01,
         1.51531622e-01,  -5.09384274e-01,  -4.31940705e-02,
        -1.73004180e-01,  -7.08071366e-02,   3.16550970e-01,
        -2.38460869e-01,  -5.43916643e-01,  -1.00490794e-01,
         2.36505479e-01,  -2.61731267e-01,   4.89670068e-01,
        -2.97301173e-01,   3.66691083e-01,  -9.42661762e-02,
        -2.99072135e-02,   1.28939077e-01,  -5.28849661e-01,
         2.78853387e-01,  -3.16180915e-01,   4.69607562e-02,
         1.79297119e-01,   5.75232744e-01,   5.56722701e-01,
         6.01089478e-01,  -2.42693841e-01,  -2.76519746e-01,
        -2.80701369e-02,   1.87477276e-01,   2.97934413e-01,
         3.90802659e-02,   5.64078391e-01,   7.19875172e-02,
         3.68543640e-02,   3.70312959e-01,  -2.42615610e-01,
        -1.95574775e-01,   4.72277731e-01,  -2.50009894e-01,
         1.91391319e-01,  -5.24945974e-01,  -2.91332118e-02,
         6.45005479e-02,   2.74381787e-01,  -1.61725581e-01,
         4.39200327e-02,

In [ ]:
sims = model.docvecs.most_similar([inferred], topn=len(model.docvecs))
sims

In [110]:
def compute_similarities(start_ind, end_ind):
    distances = []
    c = 0.0
    sum_vectors = np.zeros(500)
    for i in range(start_ind, end_ind+1):
        
        text = only_text[i]
        word2vec_vector = model.infer_vector(nltk.word_tokenize(text))
        curr_business_vector = sum_vectors / max(1.0, c)
        distances.append(cosine_similarity(word2vec_vector.reshape(1,-1), curr_business_vector.reshape(1,-1))[0,0])
        sum_vectors += word2vec_vector
        c += 1
    return distances

In [111]:
bid_to_min_ind = {}
bid_to_max_ind = {}
for i, business_index in enumerate(ordered_business_id_indices):
    if business_index_to_bid[business_index] not in bid_to_min_ind:
        bid_to_min_ind[business_index_to_bid[business_index]] = i
    bid_to_max_ind[business_index_to_bid[business_index]] = i


In [120]:
example_bid_index = 1
example_bid = business_index_to_bid[example_bid_index]

In [121]:
similarities = compute_similarities(bid_to_min_ind[example_bid], bid_to_max_ind[example_bid])
similarities

[0.0,
 0.42219237172006019,
 0.48654308404429036,
 0.68682084118800135,
 0.73490422047533599,
 0.77549705652368217,
 0.71307624746526055,
 0.70364584319310475,
 0.68133193788732327,
 0.66310232457705565,
 0.79571423946284592,
 0.80791608689687466,
 0.75883512315746604,
 0.56335725368775746,
 0.67755588058070704,
 0.80184270917098943,
 0.6925027716604415,
 0.65492170213423095,
 0.72060409280129489,
 0.64571390058204825,
 0.71371273564234272,
 0.76255301364397254,
 0.78732802923821843,
 0.66468183102833633,
 0.61684635423666467,
 0.7854697492633349,
 0.7949320970309981,
 0.63643785018547683,
 0.71865592180929072,
 0.76409203850153584,
 0.77127613786294313,
 0.66624312192692259,
 0.66375926440396626,
 0.69780915908353935,
 0.75883609007119524,
 0.67917765863273982,
 0.74622175592390172,
 0.58269463643040087,
 0.7710168489607333,
 0.79627835034279926,
 0.78098952978468339,
 0.70607619165084579,
 0.77993562682810325,
 0.75575498368878291,
 0.84373877574504519,
 0.73021955730165367,
 0.74250

In [122]:
# Create a trace
trace = Scatter(
    x = range(len(similarities)),
    y = similarities
)

data = [trace]

py.iplot(data)